# 02.08 - Sorting Arrays

Sorting algorithms are a class of algorithms with a common objective: sorting values in a list or array.

For example, here is a _selection sort_, which repeatedly finds the minimum value from a list, and makes swaps until the list is sorted:

In [1]:
import numpy as np

def selection_sort(x):
    for i in range(len(x)):
        swap = i + np.argmin(x[i:])
        (x[i], x[swap]) = (x[swap], x[i])
    return x

In [2]:
x = np.array([2, 1, 4, 3, 5])
selection_sort(x)

array([1, 2, 3, 4, 5])

Altough simple, this algorithm is not the most efficient computing in O[N^2].